In [ ]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.10:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.7/dist-packages/kaggle-1.5.10.dist-info/*
    /usr/local/lib/python3.7/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 8.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=4852978b4ea8346877a229f2cf0d4964855d30752a67164782cd2061a274c994
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
-rw-r--r-- 1 root root 65 Mar 17 01:12 kaggle.json


In [ ]:
!kaggle competitions download -c sejong-ai-challenge-p2

 64% 19.0M/29.8M [00:00<00:00, 37.7MB/s]
100% 29.8M/29.8M [00:00<00:00, 61.6MB/s]


In [ ]:
!unzip sejong-ai-challenge-p2.zip

Archive:  sejong-ai-challenge-p2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')



In [9]:
print(train.shape)
print(train['Text'][0])

(194140, 3)
Great price and quick service, the product is a back case with no front protection, but alas thats why I ordered the sleeves for the front. Seems very durable and color is very vibrant.


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194140 entries, 0 to 194139
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  194140 non-null  int64 
 1   Text        194139 non-null  object
 2   Label       194140 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


In [8]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.2 MB/s 
  Using cached pybind11-2.6.2-py2.py3-none-any.whl (191 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3096896 sha256=0c3ba091857be6eae0cb8df9bc953d805d41d438dde0125d17ab1b5f92b7bbe3
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [5]:
import fasttext

file = open('fasttexttrain.txt', 'w+')
for i in train.index:
    line = '__label__' + str(train['Label'][i]) + ' ' + str(train['Text'][i])
    file.write(line+'\n')

In [7]:
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=15, minCount=2, maxn=10, verbose=0)

In [12]:
result = text_clf_model.predict('Great price and quick service, the product is a back case with no front protection, but alas thats why I ordered the sleeves for the front. Seems very durable and color is very vibrant.', k=1)
print(result)

(('__label__1',), array([0.92144179]))


In [23]:
test['Text'].loc[7]

"See if you sound like me: - Large family with multiple devices that need charging either by Apple cables or Micro USB - Cables are always getting lost, eaten, frayed/broken, lent, or sucked into another dimensionI'm usually on Ebay or Amazon once every 3 months to buy a wholesale pack of micro USB or Apple cables because the 9+ devices we charge at home go through cables like most families go through milk.  Most cables I buy do not stand up to our abuse, and the usual break point is the cable to micro USB head.These cables from Pro Rich seem to be different.  The break point looks to be more reinforced, the cables themselves have some heft and sturdiness to them, and the LENGTH(!) is outstanding.  I would recommend these to anyone who feels like a wholesale dealer of charging cables (or anyone that seems to go through them on a regular basis)."

In [22]:
for i in test.index:
    test['Text'][i] =test['Text'][i].replace('\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:

for i in test.index:
    test_pred = text_clf_model.predict(test['Text'][i], k=1)[0][0]
    submission['Label'][i] =  test_pred.split('__')[-1]


print(submission)

          id  Label
0          0      1
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
83199  83199      0
83200  83200      0
83201  83201      0
83202  83202      0
83203  83203      1

[83204 rows x 2 columns]


In [47]:
submission.to_csv('submission.csv', index=None, header=True)

In [48]:
!kaggle competitions submit -c sejong-ai-challenge-p2 -f submission.csv -m "Message"

100% 639k/639k [00:01<00:00, 343kB/s]
Successfully submitted to Sejong AI Challenge 문제2

In [ ]:
## 0.90703